In [277]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
import time

In [286]:
news = pd.read_csv("../data/News/news.csv")
price = pd.read_csv("../data/Stock/stock.csv")

In [287]:
merge = pd.DataFrame(columns=["symbol", "date", "time","content", "preprice", "latprice", "match", "sentiment", "score"])

In [288]:
for title in ["symbol", "date", "time", "content", "match", "sentiment"]:
    merge[title] = news[title]
merge["score"] = news["match"] * news["sentiment"]

In [289]:
print(price)

            Date    meta     goog     amzn      nflx      aapl
0     2012-05-18   38.23  14.9539  10.6925    9.9943   16.1466
1     2012-05-21   34.03  15.2954  10.9055   10.2486   17.0873
2     2012-05-22   31.00  14.9639  10.7665    9.6729   16.9561
3     2012-05-23   32.00  15.1796  10.8640   10.2729   17.3698
4     2012-05-24   33.03  15.0351  10.7620   10.0386   17.2103
...          ...     ...      ...      ...       ...       ...
2641  2022-11-15  117.08  98.7200  98.9400  310.2000  150.0400
2642  2022-11-16  113.23  98.9900  97.1200  306.0200  148.7900
2643  2022-11-17  111.45  98.5000  94.8500  295.2800  150.7200
2644  2022-11-18  112.05  97.8000  94.1400  287.9800  151.2900
2645  2022-11-21  109.86  95.8300  92.4600  285.0500  148.0100

[2646 rows x 6 columns]


In [290]:
idx = 0
new = news.iloc[idx]
print(new)
symbol = new["symbol"].lower()
symbol = symbol if not symbol == "googl" else "goog"
date = new["date"]
time = new["time"]
cb = date +" "+ time
datetime_object = datetime.datetime.strptime(cb, '%Y-%m-%d %H:%M:%S')
print(datetime_object)
mv = datetime.timedelta(hours=5)
datetime_object -= mv
print(datetime_object)
if date in price.index:
    merge.iloc[idx, merge.columns.get_loc('preprice')] = price.loc[date][symbol]
    print(merge.iloc[idx]["preprice"], price.loc[date][symbol])


symbol                                                    META
date                                                2022-11-05
time                                                  20:13:24
uuid                      d5b8c1b1-a352-488a-99a0-98084f6a425f
match                                                13.746987
sentiment                                              -0.4588
content      Twitter co-founder Jack Dorsey admits he grew ...
Name: 0, dtype: object
2022-11-05 20:13:24
2022-11-05 15:13:24


In [292]:
pzmerge = pd.DataFrame()
for idx, new in tqdm(news.iterrows()):
    date = new["date"]
    time = new["time"]
    match =new["match"]
    senti = new["sentiment"]
    cb = date +" "+ time
    usdate = datetime.datetime.strptime(cb, '%Y-%m-%d %H:%M:%S')
    usdate -= datetime.timedelta(hours=5)
    symbol = new["symbol"].lower()
    symbol = symbol if not symbol == "googl" else "goog"
    dtformat = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    while True:
        if price['Date'].eq(date).any():
            # print(date, price['Date'].eq(date).any())
            break
        else:
            dtformat+=datetime.timedelta(days=1)
            date = datetime.datetime.strftime(dtformat,'%Y-%m-%d')
    row = price[price["Date"]==date]
    if usdate.hour<16:
        # print(usdate.hour,"<16")[["Date","aapl"]]
        pz1 = price.iloc[row.index][[symbol]]
        pz0 = price.iloc[row.index-1][[symbol]]
        pz2 = price.iloc[row.index+1][[symbol]]
    else:
        # print(usdate.hour,">=16")
        pz0 = price.iloc[row.index][[symbol]]
        pz1 = price.iloc[row.index+1][[symbol]]
        pz2 = price.iloc[row.index+2][[symbol]]
    # pz0 = pz0.to_frame()
    # pz1 = pz1.to_frame()
    # pz2 = pz2.to_frame()
    pz = pd.concat([pz0,pz1,pz2]).T
    pz.columns = ["pz0","pz1","pz2"]
    # print([pz0,pz1,pz2])
    pzmerge = pd.concat([pzmerge, pz])

# print(pzmerge)

25278it [00:33, 744.11it/s]

         pz0     pz1     pz2
meta   90.79   96.72   96.47
meta   90.79   96.72   96.47
meta   90.79   96.72   96.47
meta   90.79   96.72   96.47
meta   90.79   96.72   96.47
...      ...     ...     ...
nflx  579.84  565.17  556.78
nflx  579.84  565.17  556.78
nflx  579.84  565.17  556.78
nflx  579.84  565.17  556.78
nflx  579.84  565.17  556.78

[25278 rows x 3 columns]


In [334]:
df = news.reset_index(drop=True)
df1 = pzmerge.reset_index(drop=True)
merge = pd.concat([df,df1], axis = 1)
print(merge)
merge.to_csv("../data/merge2.csv")

      symbol        date      time                                  uuid  \
0       META  2022-11-05  20:13:24  d5b8c1b1-a352-488a-99a0-98084f6a425f   
1       META  2022-11-05  18:02:00  186d471c-27a8-4e3d-a905-ab6982b32996   
2       META  2022-11-05  18:00:00  a6e7fdb2-80ff-46ea-bd07-53f5c6416111   
3       META  2022-11-05  16:49:01  5ff55fac-7139-46f1-bc32-30ca00379954   
4       META  2022-11-05  11:58:00  3291b5cf-5c58-4a63-a1c0-db633b9ba218   
...      ...         ...       ...                                   ...   
25273   NFLX  2021-01-22  14:55:08  2e8daa5d-b22e-43de-93b1-03ef99e57857   
25274   NFLX  2021-01-22  14:48:47  df63c55b-0ed3-432c-aa66-69911d9986e7   
25275   NFLX  2021-01-22  14:42:59  a3db3a06-5a21-4dd7-b42b-63fbfbd48a49   
25276   NFLX  2021-01-22  13:55:04  e9447691-d953-4025-8d04-2cdf8aaa0aa7   
25277   NFLX  2021-01-22  13:30:00  68dfe285-a462-4765-97f9-e261b177c24a   

           match  sentiment  \
0      13.746987  -0.458800   
1       9.408848   0.6249